## Run Grid Search on Machine Learning Algorithms

In [1]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
%run ml_helpers.ipynb

In [3]:
#Import bucketized data

#Run convert_to_categorical
#data = ml_helpers.convert_to_categorical(data, [columns_to_convert])

#Run prep_data(df, y, num_years, year_col, vars_to_onehot):
#data_list = ....

#If bucketized data doesn't have spatial lag, make it have spatial lag

In [4]:
def search_best_results(X_train, y_train, X_test, y_test, models, grid):
    '''
    Returns accuracy measures for model with different hyperparameters 
    '''
    results = pd.DataFrame(columns = ["parameters", "precision"])
    
    for model_key in models.keys():         
        # Loop over parameters 
        for params in grid[model_key]: 
            
            # Create model 
            model = models[model_key]
            model.set_params(**params)
            
            # Fit model on training set 
            model.fit(X_train, y_train)
            
            # Predict on testing set 
            y_pred = model.predict(X_test)
            
            # Evaluate predictions 
            precision = sk.metrics.precision_score(y_test, y_pred),
            precision=precision[0]
            # Store results in your results data frame 
            results2 = pd.DataFrame([[params, precision]], 
                                    columns = ["parameters", 
                                                "precision"])
            results = results.append(results2)
    return results

In [5]:
def evaluate_results(X_train, y_train, X_test, y_test, models, grid):
    '''
    Returns precision and recall for model 
    This function should be used only for final data frame 
    '''
    results = pd.DataFrame(columns = ["parameters", "precision", "recall"])
    
    for model_key in models.keys():         
        # Loop over parameters 
        for params in grid[model_key]: 
            
            # Create model 
            model = models[model_key]
            model.set_params(**params)
            
            # Fit model on training set 
            model.fit(X_train, y_train)
            
            # Predict on testing set 
            y_pred = model.predict(X_test)
            
            # Evaluate predictions 
            precision = sk.metrics.precision_score(y_test, y_pred),
            recall = sk.metrics.recall_score(y_test, y_pred),
            print("precision is:", precision, "|recall is:", recall)
            precision=precision[0]
            recall = recall[0]
            # Store results in your results data frame 
            results2 = pd.DataFrame([[params, precision, recall]], 
                                    columns = ["parameters", 
                                                "precision", "recall"])
            results = results.append(results2)
    return results

In [6]:
def average_grid_searches(model, model_name, 
                          grid, 
                          data_list, y, test_year):
    '''
    Model: sklearn model object
    model_name (string): a name for the model
    grid (dict): maps hyperparameters to potential options:
    scoring (list of strings): a list of the ways to score
    data (pd.DataFrame): a pandas dataframe with all the data
    test_year (int): The year we seek to predict
    
    Idea to concatenate rows of dfs came via the first answer on from this stack exchange: 
    https://stackoverflow.com/questions/44515888/compute-average-mean-across-dataframes-in-python-pandas
    
    Get mean of col rows help from here: 
    https://stackoverflow.com/questions/33750326/compute-row-average-in-pandas
    
    grid example: {model_name: [{'penalty': x, 'C': y, 'random_state': 0} 
                           for x in ('l2', 'none') \
                           for y in (0.01, 0.1, 1, 10, 100)]}
    
    '''
    print("test year is:", test_year)
    target_years_in_train = list(np.arange(2015,test_year))
    target_years_in_train.pop(0)
    print(target_years_in_train)
    results_dfs = []
    
    for data_set in data_list:
        train_df = data_set[0]
        test_df = data_set[1]
        year = data_set[2]
        X_train, y_train = split_X_y(train_df, y)
        X_test, y_test = split_X_y(test_df, y)
        
        
        models = {model_name: model}
        
        results_df = search_best_results(X_train, y_train, X_test, y_test, models, grid)
        results_df = results_df.add_suffix("_" + str(year))
        results_dfs.append(results_df)
    
    all_results = pd.concat(results_dfs, axis = 1)
    all_results["mean"] = all_results.mean(axis=1, numeric_only=True)
    #all_results = all_results[["params_" + str(test_year-1), "mean"]]
    all_results.sort_values("mean", inplace=True, ascending=False)
    #results_to_return = all_results["params", 'mean'].sort_values("mean")
    #return results_to_return

    #log_reg_results["mean"]=log_reg_results.mean(axis=1, numeric_only=True)


    
    return all_results

In [7]:
def results_by_year(model, model_name, 
                          grid, 
                          data_list, y, test_year):
    '''
    Model: sklearn model object
    model_name (string): a name for the model
    grid (dict): maps hyperparameters to potential options:
    scoring (list of strings): a list of the ways to score
    data (pd.DataFrame): a pandas dataframe with all the data
    test_year (int): The year we seek to predict
    
    Idea to concatenate rows of dfs came via the first answer on from this stack exchange: 
    https://stackoverflow.com/questions/44515888/compute-average-mean-across-dataframes-in-python-pandas
    
    Get mean of col rows help from here: 
    https://stackoverflow.com/questions/33750326/compute-row-average-in-pandas
    
    grid example: {model_name: [{'penalty': x, 'C': y, 'random_state': 0} 
                           for x in ('l2', 'none') \
                           for y in (0.01, 0.1, 1, 10, 100)]}
    
    '''
    print("test year is:", test_year)
    target_years_in_train = list(np.arange(2015,test_year))
    target_years_in_train.pop(0)
    print(target_years_in_train)
    results_dfs = []
    
    for data_set in data_list:
        train_df = data_set[0]
        test_df = data_set[1]
        year = data_set[2]
        X_train, y_train = split_X_y(train_df, y)
        X_test, y_test = split_X_y(test_df, y)
        
        
        models = {model_name: model}
        
        results_df = evaluate_results(X_train, y_train, X_test, y_test, models, grid)
        results_df = results_df.add_suffix("_" + str(year))
        results_dfs.append(results_df)
    
    all_results = pd.concat(results_dfs, axis = 1)

    return all_results

### Test the code:

In [8]:
#Get data set up
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv("../intermediate_data/df_2015_to_present.csv")
data["was_arrested"]=data["Arrest"].astype("float")
data = data.drop(["Arrest", "category_1", "category_2", "Domestic", "ID", "Week", "Day", "Hour"], axis = 1)
data = convert_to_categorical(data, ["Beat"])
data_small = data.sample(frac=0.001)

data_small_ready = prep_data(data_small, "was_arrested",
                                        1, "Year", ["Year", "Month", "Beat", "Watch"])

         Unnamed: 0  Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  \
1226845     1226845  1215  2019      8   Third  0.00   0.0    91    71   
1300240     1300240   223  2019     11   First  0.00   0.0    38    22   
1268495     1268495   323  2019      9   Third  0.99   0.0    69    62   
1166182     1166182  1214  2019      5   Third  0.18   0.0    56    46   
1326733     1326733   111  2019     12   Third  0.00   0.0    53    36   
...             ...   ...   ...    ...     ...   ...   ...   ...   ...   
1285148     1285148  1935  2019     10   Third  0.01   0.0    52    45   
1263141     1263141  2534  2019      9  Second  0.74   0.0    78    67   
1149559     1149559  1533  2019      4   Third  1.51   0.0    53    43   
1091555     1091555   722  2019      2   First  0.00   0.0    41    15   
1131774     1131774  2223  2019      4   First  0.37   0.0    48    40   

         count_l_stops  ...  count_restaurants  count_bars  count_daycares  \
1226845            1.0  ...      

Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [10]:
#train = data_small_ready[0][0]
#test = data_small_ready[0][1]
#train_X, train_y = split_X_y(train, "was_arrested")
#test_X, test_y = split_X_y(test, "was_arrested")
#models = {
#    'LogisticRegression': LogisticRegression(), 
#}
#
#grid = {
#    'LogisticRegression': [{'penalty': x, 'C': y, 'random_state': 0} 
#                           for x in ('l2', 'none') \
#                           for y in (0.01, 0.1, 1, 10, 100)]
#}
#res = search_best_results(train_X, train_y, test_X, test_y, models, grid)
#res